In [1]:
import pandas as pd
from sklearn.preprocessing import Normalizer, MinMaxScaler, StandardScaler, RobustScaler,QuantileTransformer, PowerTransformer
from sklearn.compose import ColumnTransformer
import keras

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import os

import math


from tensorflow.keras.utils import Sequence
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import datetime as dt

import warnings
warnings.filterwarnings('ignore')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)  # turn off deprecation warnings



2023-04-16 00:16:07.431079: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-16 00:16:07.494240: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-16 00:16:07.495268: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-16 00:16:08.439062: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('../data/final_dataset22.csv')
df

,datetime,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,...,NI,RICE,XAU,IRD,SUGAR,ETHANOL,CORN,COTTON,TIN,ZNC
0,1641017460000,47249.44,47281.68,47249.44,47271.88,10.37408,1641017519999,4.903711e+05,403,6.99871,...,0.62547,14.632537,0.000547,4147.926037,0.189924,2.136423,5.928399,1.129187,1.222731,0.102048
1,1641017520000,47271.87,47290.00,47271.87,47289.97,4.11304,1641017579999,1.944688e+05,343,2.50207,...,0.62547,14.632537,0.000547,4147.926037,0.189924,2.136423,5.928399,1.129187,1.222731,0.102048
2,1641017580000,47289.96,47320.00,47287.60,47300.72,13.45235,1641017639999,6.363889e+05,472,6.83725,...,0.62547,14.632537,0.000547,4147.926037,0.189924,2.136423,5.928399,1.129187,1.222731,0.102048
3,1641017640000,47300.71,47324.42,47293.01,47307.04,6.17338,1641017699999,2.920746e+05,416,3.06969,...,0.62547,14.632537,0.000547,4147.926037,0.189924,2.136423,5.928399,1.129187,1.222731,0.102048
4,1641017700000,47307.03,47307.04,47258.73,47259.14,4.41853,1641017759999,2.088964e+05,415,1.66964,...,0.62547,14.632537,0.000547,4147.926037,0.189924,2.136423,5.928399,1.129187,1.222731,0.102048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525525,1672548960000,16521.42,16521.93,16516.49,16518.76,83.64312,1672549019999,1.381688e+06,2545,41.92976,...,29975.00000,18.000000,0.000548,4800.000000,0.200500,2.161000,6.785000,0.830000,19391.166190,3012.500000
525526,1672549020000,16518.76,16519.22,16517.03,16517.41,48.86392,1672549079999,8.071365e+05,1783,21.38648,...,29975.00000,18.000000,0.000548,4800.000000,0.200500,2.161000,6.785000,0.830000,19391.166190,3012.500000
525527,1672549080000,16517.39,16520.64,16517.28,16519.37,61.07865,1672549139999,1.008950e+06,2350,36.47444,...,29975.00000,18.000000,0.000548,4800.000000,0.200500,2.161000,6.785000,0.830000,19391.166190,3012.500000
525528,1672549140000,16519.37,16522.81,16511.92,16521.85,173.39683,1672549199999,2.864058e+06,4050,89.59732,...,29975.00000,18.000000,0.000548,4800.000000,0.200500,2.161000,6.785000,0.830000,19391.166190,3012.500000


In [3]:
df = df.drop(columns = ['ignore','datetime'])
df_copy = df.copy()


In [4]:


# delay: time in future that will be predicted
delay = 4 * 60


# sampling rate: period between timesteps within the sequence
# Sequence with rate=1 : t1,t2...tn
# Sequence with rate=3 : t1,t3...tn*3
sampling_rate = 30

# sequence length: sequence lenght of each sample
half_day = 12 * 60
sequence_length = int(6 * half_day / sampling_rate)


# sequence_stride: period between sequences
# First sequence starts at t0
# Second sequence will start at t1 with sequence_stride=1 or at t5 with sequence_stride=5
sequence_stride = 1



#batch_size: Number of timeseries samples in each batch (except maybe the last one). 
#If None, the data will not be batched (the dataset will yield individual samples).
# Huge impact in performance.
# Tip, should be multiple of 8
batch_size = 64

# Understanding our parameters
msg = f"The timeseries will consist of batches containing {batch_size} sequences of {sequence_length} samples."

msg += f"\nFinally our target is {delay} timesteps in the future"
print(msg)

The timeseries will consist of batches containing 64 sequences of 144 samples.
Finally our target is 240 timesteps in the future


In [5]:
#import matplotlib.pyplot as plt
#loss = history.history["mae"]
#val_loss = history.history["val_mae"]
#epochs = range(1, len(loss) + 1)
#plt.figure()
#plt.plot(epochs, loss, "bo", label="Training MAE")
#plt.plot(epochs, val_loss, "b", label="Validation MAE")
#plt.title("Training and validation MAE")
#plt.legend()
#plt.show()


In [ ]:
results = []

# Create a list of scalers
scalers = [('standard', StandardScaler()), 
           ('minmax', MinMaxScaler()), 
           ('robust', RobustScaler()), 
           ('quantile', QuantileTransformer()), 
           ('power', PowerTransformer()), 
           ('normalize', Normalizer())]

# Define a pipeline for each scaler
for scaler_name, scaler in scalers:
    print(scaler_name)

    df = df_copy.copy()

    # Fit the scaler on your dataframe (let's say it's called df)
    df_normalized = scaler.fit_transform(df)

    # Convert the normalized data back to a dataframe
    df_normalized = pd.DataFrame(df_normalized, columns=df.columns)

    # Convert the DataFrame to a NumPy array
    raw_data = df_normalized.values
    close = df_normalized['close'].values
    close = close.reshape((len(close),1))
    # Display the NumPy array
    print(type(raw_data))
    print(raw_data.shape)
    print(close.shape)
    
    print('Prep Data')
    num_train_samples = int(0.6 * len(raw_data))
    num_val_samples = int(0.25 * len(raw_data))
    num_test_samples = len(raw_data) - num_train_samples - num_val_samples
    print("num_train_samples:", num_train_samples)
    print("num_val_samples:", num_val_samples)
    print("num_test_samples:", num_test_samples)

    

    train_dataset = keras.preprocessing.timeseries_dataset_from_array(
                        raw_data[:-delay],
                        targets=close[delay:],
                        sampling_rate=sampling_rate,
                        sequence_stride=sequence_stride,
                        sequence_length=sequence_length,
                        shuffle=False, # Shouldn't the shuffle be set to 0?
                        seed=33,
                        batch_size=batch_size,
                        start_index=0,
                        end_index=num_train_samples)

    print("Done Train")

    val_dataset = keras.preprocessing.timeseries_dataset_from_array(
                        raw_data[:-delay],
                        targets=close[delay:],
                        sampling_rate=sampling_rate,
                        sequence_stride=sequence_stride,
                        sequence_length=sequence_length,
                        shuffle=False,
                        seed=33,
                        batch_size=batch_size,
                        start_index=num_train_samples,
                        end_index=num_train_samples + num_val_samples)

    print("Done Validation")      
        
    test_dataset = keras.preprocessing.timeseries_dataset_from_array(
                        raw_data[:-delay],
                        targets=close[delay:],
                        sampling_rate=sampling_rate,
                        sequence_stride=sequence_stride,
                        sequence_length=sequence_length,
                        shuffle=False,
                        seed=33,
                        batch_size=batch_size,
                        start_index=num_train_samples + num_val_samples)

    print("Done Test")

    print('Training RNN')####################################################################
    inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
    x = layers.SimpleRNN(32)(inputs)
    outputs = layers.Dense(1)(x)
    model = keras.Model(inputs, outputs)

    callbacks = [
        keras.callbacks.ModelCheckpoint("models/simple_rnn",
    save_best_only=True)
    ]

    model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

    history = model.fit(train_dataset,
                        epochs=10,
                        validation_data=val_dataset,
                        callbacks=callbacks)
    
    loss = history.history["mae"]
    val_loss = history.history["val_mae"]
    epochs = range(1, len(loss) + 1)
    results.append([scaler_name, 'RNN', loss, val_loss, epochs])
    
    print('Training LSTM')####################################################################
    x = layers.LSTM(32)(inputs)
    outputs = layers.Dense(1)(x)
    model = keras.Model(inputs, outputs)

    callbacks = [
        keras.callbacks.ModelCheckpoint("models/lstm",
       save_best_only=True) 
    ]

    model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

    history = model.fit(train_dataset,
                        epochs=10,
                        validation_data=val_dataset,
                        callbacks=callbacks)
    
    #lstm.compile(optimizer='rmsprop',
                  #loss=['mse', 'mae'],
                  #metrics=['accuracy', 'mse', 'mae'])
    
    loss = history.history["mae"]
    val_loss = history.history["val_mae"]
    epochs = range(1, len(loss) + 1)
    results.append([scaler_name, 'LSTM', loss, val_loss, epochs])
    
    
    
    loss = history.history["mae"]
    val_loss = history.history["val_mae"]
    epochs = range(1, len(loss) + 1)
    results.append([scaler_name, 'BiLSTM', loss, val_loss, epochs])

    

In [10]:
# Open a text file in write mode
with open('results.txt', 'w') as file:
    # Iterate through the list and write each item to a new line in the file
    for item in results:
        file.write(str(item) + '\n')

In [11]:
results

[['standard',
  'RNN',
  [0.03656119108200073,
   0.024024421349167824,
   0.023402078077197075,
   0.023441726341843605,
   0.023609261959791183,
   0.023657100275158882,
   0.02357877604663372,
   0.023555949330329895,
   0.023506255820393562,
   0.023469481617212296],
  [0.2753670811653137,
   0.27057403326034546,
   0.26838499307632446,
   0.267702579498291,
   0.2670438885688782,
   0.2660065293312073,
   0.2653213143348694,
   0.26513412594795227,
   0.26456600427627563,
   0.2642897963523865],
  range(1, 11)],
 ['standard',
  'LSTM',
  [0.030144261196255684,
   0.023986490443348885,
   0.020855287089943886,
   0.020299961790442467,
   0.019880009815096855,
   0.019756555557250977,
   0.0199420303106308,
   0.019437531009316444,
   0.01906595565378666,
   0.018766416236758232],
  [0.48405730724334717,
   0.2959347367286682,
   0.3930286765098572,
   0.46193841099739075,
   0.5005449652671814,
   0.45721206068992615,
   0.4072936177253723,
   0.4978668987751007,
   0.4673292338848

In [18]:
lista = [['standard',
  'RNN',
  0.023469481617212296,
  0.2642897963523865,
  range(1, 11)],
 ['standard',
  'LSTM',
  0.018766416236758232,
  0.4687740206718445,
  range(1, 11)],
 ['standard',
  'BiLSTM',
  0.01800471358001232,
  0.32036086916923523,
  range(1, 11)],
 ['minmax',
  'RNN',
  0.015271274372935295,
  0.08089461922645569,
  range(1, 11)],
 ['minmax',
  'LSTM',
  0.007987561635673046,
  0.057518262416124344,
  range(1, 11)],
 ['minmax',
  'BiLSTM',
  0.0058118849992752075,
  0.05520883947610855,
  range(1, 11)],
 ['robust',
  'RNN',
  0.016051141545176506,
  0.17699980735778809,
  range(1, 11)],
 ['robust',
  'LSTM',
  0.011219332925975323,
  0.05622795969247818,
  range(1, 11)]]

In [20]:
import pandas as pd

df = pd.DataFrame(lista)
df.drop(columns= 4, inplace=True)
df.columns = ['Normalization' , 'Model', 'Train MAE', 'Test MAE']

df['MAE Index'] = 100 * df['Train MAE'] * df['Test MAE']
df.sort_values(by='MAE Index')

,Normalization,Model,Train MAE,Test MAE,MAE Index
5,minmax,BiLSTM,0.005812,0.055209,0.032087
4,minmax,LSTM,0.007988,0.057518,0.045943
7,robust,LSTM,0.011219,0.056228,0.063084
3,minmax,RNN,0.015271,0.080895,0.123536
6,robust,RNN,0.016051,0.177000,0.284105
2,standard,BiLSTM,0.018005,0.320361,0.576801
0,standard,RNN,0.023469,0.264290,0.620274
1,standard,LSTM,0.018766,0.468774,0.879721
